# PyCity Schools Analysis

Analysis of Means

The primary assessment in the PyCitySchool notebook is the per_school_summary table. This table represents the overarching scale of schools within the school district. Percent Passing columns are our critical performance indicators for each school. Additionally, the spending_summary, size_summary, and type_summary are additive to change our view to a specific perspective. Those perspectives are excellent since they remove other factors that might pull away from a desired perspective.

My first conclusion draws on analyzing our overall worst-performing and best-performing schools. The top-performing schools are all charter schools. There could be many reasons for this, but the instance represented with this current data set is the total number of students per school. Using the School Size table, we can surmise that larger school populations lead to lower scores; we reinforce this notion in the bottom_schools table, where, compared to top-performing schools, the total number of students is much larger. That goes hand-in-hand with charter versus public schools; charter schools often cap the number of students; smaller class sizes or cohorts lead to less strained instructor interactions.

My second conclusion draws from the spending Ranges (Per Student) summary. In this perspective, money is not a direct factor in the score effect. Instead, it is students who have less money to spend that are performing the best. That feeds into the psychological premise that limitation forces creativity; with less around, students need to be creative in their approach to learning. Constraints help define scope, which helps achieve an overall goal.
  


In [1]:
# import packages and give them cool setup names
import pandas as pd
from pathlib import Path
import numpy as np

In [2]:
# Create and store filepath, include wd note in readme
school_csv = Path("Resources/schools_complete.csv") # pathlib autocomplete if initial is correct
student_csv = Path("Resources/students_complete.csv")

# refile each csv as a dataframe
school_df = pd.read_csv(school_csv)
student_df = pd.read_csv(student_csv)

In [3]:
# merge data frames, using how = 'left', such that the student frame is prioritized, unmatched rows found in school frame yields NaN
student_school_df = pd.merge(student_df,school_df,how = "left", on = ["school_name", "school_name"])
# check that it exists, Dr.Richard what a great first name
student_school_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [4]:
# number of unique schools
skewl_ct = student_school_df['school_name'].nunique()
# total number of students
student_ct = student_school_df['student_name'].count()
# district budget
budget_tot = school_df['budget'].sum()
# mean math
math_mean = student_school_df['math_score'].mean()
# mean reading
reading_mean = student_school_df['reading_score'].mean()

# percentage of students who passed math (threshold 70)
passing_math_count = student_school_df[
    (student_school_df["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_ct) * 100

# same for reading
passing_reading_count = student_school_df[
    (student_school_df["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_ct) * 100

# overall rate, combined
passing_math_reading_count = student_school_df[
    (student_school_df["math_score"] >= 70) & (student_school_df["reading_score"] >= 70)].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_ct) * 100

In [5]:
# creating a snapshot dictionary of the above, formatting based on instruction
district = {
    'Total Schools': [skewl_ct],
    'Total Students': [student_ct],
    'Total Budget': [budget_tot],
    'Avg Math Score': [math_mean],
    'Avg Reading Score': [reading_mean],
    '% Passing Math': [passing_math_percentage],
    '% Passing Reading': [passing_reading_percentage],
    '% Overall Passing': [overall_passing_rate]
}
district_summary = pd.DataFrame(district)
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [6]:
# similar deal, using groupby to aggregate by school_name and then count/sum
base_school_df = student_school_df.groupby(['school_name'])
school_type = pd.DataFrame(base_school_df['type'].unique())
# turns out you can compute the mean for all the numerics directly from base_school_df, similar to the Capmoulin means
# although weirdly, the mean in some instances is equal to what the count/sum
base_school_means_df = round(pd.DataFrame(base_school_df.mean(numeric_only=True)),3)
base_school_means_df.reset_index()
# selecting the columns we want before we continue
base_school_means_df = base_school_means_df[['reading_score', 'math_score', 'size', 'budget']]

In [7]:
# filtering student_school df for passing scores
# can't do it all in one unfortunately, columns would be uneven
pass_overall_df = student_school_df[(student_school_df['math_score'] >= 70) & (student_school_df['reading_score'] >= 70)]
pass_math_df = student_school_df[(student_school_df['math_score'] >= 70)]
pass_reading_df = student_school_df[(student_school_df['reading_score'] >= 70)]

# note the df that's being fed into groupby, differentiating based on what we filtered before hand
pass_overall_groupby = pd.DataFrame(pass_overall_df.groupby('school_name')['student_name'].count())
pass_math_groupby = pd.DataFrame(pass_math_df.groupby('school_name')['student_name'].count())
pass_reading_groupby = pd.DataFrame(pass_reading_df.groupby('school_name')['student_name'].count())

In [8]:
# concat all our jazz
per_school_summary = pd.concat([base_school_means_df,school_type,pass_reading_groupby,pass_math_groupby,pass_overall_groupby],axis=1)

# renaming columns the dangerous/efficient way
per_school_summary.columns = ['Avg Reading Score', 'Avg Math Score', 'Total Students', 'School Budget', 'School Type', 'Pass Reading Count', 'Pass Math Count', 'Pass Overall Count' ]

# settle the % passing calc and add as new columns
per_school_summary['% Passing Reading'] = round((per_school_summary['Pass Reading Count'])/(per_school_summary['Total Students'])*100, 3)
per_school_summary['% Passing Math'] = round((per_school_summary['Pass Math Count'])/(per_school_summary['Total Students'])*100, 3)
per_school_summary['% Passing Overall'] = round((per_school_summary['Pass Overall Count'])/(per_school_summary['Total Students'])*100, 3)
# per capita calc
per_school_summary['Per Student Budget'] = (per_school_summary['School Budget'])/(per_school_summary['Total Students'])

# trim down columns and reorder the whole thing
trim = ['School Type', 'Total Students', 'School Budget', 'Per Student Budget', 'Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Passing Overall']
per_school_summary = per_school_summary[trim]
# adding the recommended formatting
per_school_summary["School Budget"] = per_school_summary["School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)
per_school_summary.index.name = None
per_school_summary 

,School Type,Total Students,School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Bailey High School,[District],4976.0,"$3,124,928.00",$628.00,77.048,81.034,66.680,81.933,54.642
Cabrera High School,[Charter],1858.0,"$1,081,356.00",$582.00,83.062,83.976,94.133,97.040,91.335
Figueroa High School,[District],2949.0,"$1,884,411.00",$639.00,76.712,81.158,65.988,80.739,53.204
Ford High School,[District],2739.0,"$1,763,916.00",$644.00,77.103,80.746,68.310,79.299,54.290
Griffin High School,[Charter],1468.0,"$917,500.00",$625.00,83.351,83.817,93.392,97.139,90.599
Hernandez High School,[District],4635.0,"$3,022,020.00",$652.00,77.290,80.934,66.753,80.863,53.528
Holden High School,[Charter],427.0,"$248,087.00",$581.00,83.803,83.815,92.506,96.253,89.227
Huang High School,[District],2917.0,"$1,910,635.00",$655.00,76.629,81.183,65.684,81.316,53.514
Johnson High School,[District],4761.0,"$3,094,650.00",$650.00,77.072,80.966,66.058,81.222,53.539
Pena High School,[Charter],962.0,"$585,858.00",$609.00,83.840,84.045,94.595,95.946,90.541


## Highest-Performing Schools (by % Overall Passing)

In [9]:
# top 5 schools by % overall passing
top_schools = per_school_summary.sort_values(by='% Passing Overall', ascending=False)
top_schools.index.name = None
top_schools.head(5)

,School Type,Total Students,School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Cabrera High School,[Charter],1858.0,"$1,081,356.00",$582.00,83.062,83.976,94.133,97.040,91.335
Thomas High School,[Charter],1635.0,"$1,043,130.00",$638.00,83.418,83.849,93.272,97.309,90.948
Griffin High School,[Charter],1468.0,"$917,500.00",$625.00,83.351,83.817,93.392,97.139,90.599
Wilson High School,[Charter],2283.0,"$1,319,574.00",$578.00,83.274,83.989,93.868,96.540,90.583
Pena High School,[Charter],962.0,"$585,858.00",$609.00,83.840,84.045,94.595,95.946,90.541


In [10]:
# bottom 5 schools by % overall passing
bottom_schools = per_school_summary.sort_values(by='% Passing Overall', ascending=True)
bottom_schools.index.name = None
bottom_schools.head()

,School Type,Total Students,School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Rodriguez High School,[District],3999.0,"$2,547,363.00",$637.00,76.843,80.745,66.367,80.220,52.988
Figueroa High School,[District],2949.0,"$1,884,411.00",$639.00,76.712,81.158,65.988,80.739,53.204
Huang High School,[District],2917.0,"$1,910,635.00",$655.00,76.629,81.183,65.684,81.316,53.514
Hernandez High School,[District],4635.0,"$3,022,020.00",$652.00,77.290,80.934,66.753,80.863,53.528
Johnson High School,[District],4761.0,"$3,094,650.00",$650.00,77.072,80.966,66.058,81.222,53.539


## Math Scores by Grade

In [11]:
# code provided to separate data by grade
ninth_graders = student_school_df[(student_school_df["grade"] == "9th")]
tenth_graders = student_school_df[(student_school_df["grade"] == "10th")]
eleventh_graders = student_school_df[(student_school_df["grade"] == "11th")]
twelfth_graders = student_school_df[(student_school_df["grade"] == "12th")]

# groupby school_name, use mean math_score for each
ninth_grader_math_scores = pd.DataFrame(ninth_graders.groupby('school_name')['math_score'].mean())
tenth_grader_math_scores = pd.DataFrame(tenth_graders.groupby('school_name')['math_score'].mean())
eleventh_grader_math_scores = pd.DataFrame(eleventh_graders.groupby('school_name')['math_score'].mean())
twelfth_grader_math_scores = pd.DataFrame(twelfth_graders.groupby('school_name')['math_score'].mean())

# concat all four
math_scores_by_grade = pd.concat([ninth_grader_math_scores,tenth_grader_math_scores,eleventh_grader_math_scores,twelfth_grader_math_scores],axis=1)
math_scores_by_grade.columns = ["9th", "10th", "11th", "12th"]
# is this wrangling?
math_scores_by_grade.index.name = None
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [12]:
# same process as math
ninth_graders = student_school_df[(student_school_df["grade"] == "9th")]
tenth_graders = student_school_df[(student_school_df["grade"] == "10th")]
eleventh_graders = student_school_df[(student_school_df["grade"] == "11th")]
twelfth_graders = student_school_df[(student_school_df["grade"] == "12th")]

# groupby school_name, use mean math_score for each
ninth_grader_reading_scores = pd.DataFrame(ninth_graders.groupby('school_name')['reading_score'].mean())
tenth_grader_reading_scores = pd.DataFrame(tenth_graders.groupby('school_name')['reading_score'].mean())
eleventh_grader_reading_scores = pd.DataFrame(eleventh_graders.groupby('school_name')['reading_score'].mean())
twelfth_grader_reading_scores = pd.DataFrame(twelfth_graders.groupby('school_name')['reading_score'].mean())

# concat all four
reading_scores_by_grade = pd.concat([ninth_grader_reading_scores,tenth_grader_reading_scores,eleventh_grader_reading_scores,twelfth_grader_reading_scores],axis=1)
reading_scores_by_grade.columns = ["9th", "10th", "11th", "12th"]
# is this wrangling?
reading_scores_by_grade.index.name = None
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [13]:
# suggested bins to use
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [14]:
# returning columns from whence they came, otherwise binning is a no go
per_school_summary['School Budget'] = per_school_summary['School Budget'].str.replace('$', '').str.replace(',', '').astype(float)
per_school_summary['Per Student Budget'] = per_school_summary['Per Student Budget'].str.replace('$', '').str.replace(',', '').astype(float)
# copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [15]:
# using pd.cut to sort into bins, feeding those values into a new column
school_spending_df['Spending Ranges (Per Student)'] = pd.cut(school_spending_df['Per Student Budget'], bins=spending_bins, labels=labels)
# formatting returns
school_spending_df["School Budget"] = school_spending_df["School Budget"].map("${:,.2f}".format)
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].map("${:,.2f}".format)
school_spending_df

,School Type,Total Students,School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall,Spending Ranges (Per Student)
Bailey High School,[District],4976.0,"$3,124,928.00",$628.00,77.048,81.034,66.680,81.933,54.642,$585-630
Cabrera High School,[Charter],1858.0,"$1,081,356.00",$582.00,83.062,83.976,94.133,97.040,91.335,<$585
Figueroa High School,[District],2949.0,"$1,884,411.00",$639.00,76.712,81.158,65.988,80.739,53.204,$630-645
Ford High School,[District],2739.0,"$1,763,916.00",$644.00,77.103,80.746,68.310,79.299,54.290,$630-645
Griffin High School,[Charter],1468.0,"$917,500.00",$625.00,83.351,83.817,93.392,97.139,90.599,$585-630
Hernandez High School,[District],4635.0,"$3,022,020.00",$652.00,77.290,80.934,66.753,80.863,53.528,$645-680
Holden High School,[Charter],427.0,"$248,087.00",$581.00,83.803,83.815,92.506,96.253,89.227,<$585
Huang High School,[District],2917.0,"$1,910,635.00",$655.00,76.629,81.183,65.684,81.316,53.514,$645-680
Johnson High School,[District],4761.0,"$3,094,650.00",$650.00,77.072,80.966,66.058,81.222,53.539,$645-680
Pena High School,[Charter],962.0,"$585,858.00",$609.00,83.840,84.045,94.595,95.946,90.541,$585-630


In [16]:
#  given worksheet calcs
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Avg Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Avg Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Overall"].mean()

# assemble into df
spending_summary = pd.concat([spending_math_scores,spending_reading_scores,spending_passing_math,spending_passing_reading, overall_passing_spending], axis=1)
spending_summary

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Ranges (Per Student),,,,,
<$585,83.45525,83.933750,93.46000,96.611000,90.3695
$585-630,81.89950,83.155500,87.13350,92.718250,81.4185
$630-645,78.51900,81.624500,73.48425,84.391750,62.8575
$645-680,76.99700,81.027667,66.16500,81.133667,53.5270


## Scores by School Size

In [17]:
# suggested bins
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# add new col that indicates school size, attribute and sort base on above labels, no copies here for extra danger
per_school_summary['School Size'] = pd.cut(school_spending_df['Total Students'], bins=size_bins, labels=labels)
per_school_summary

,School Type,Total Students,School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall,School Size
Bailey High School,[District],4976.0,3124928.0,628.0,77.048,81.034,66.680,81.933,54.642,Large (2000-5000)
Cabrera High School,[Charter],1858.0,1081356.0,582.0,83.062,83.976,94.133,97.040,91.335,Medium (1000-2000)
Figueroa High School,[District],2949.0,1884411.0,639.0,76.712,81.158,65.988,80.739,53.204,Large (2000-5000)
Ford High School,[District],2739.0,1763916.0,644.0,77.103,80.746,68.310,79.299,54.290,Large (2000-5000)
Griffin High School,[Charter],1468.0,917500.0,625.0,83.351,83.817,93.392,97.139,90.599,Medium (1000-2000)
Hernandez High School,[District],4635.0,3022020.0,652.0,77.290,80.934,66.753,80.863,53.528,Large (2000-5000)
Holden High School,[Charter],427.0,248087.0,581.0,83.803,83.815,92.506,96.253,89.227,Small (<1000)
Huang High School,[District],2917.0,1910635.0,655.0,76.629,81.183,65.684,81.316,53.514,Large (2000-5000)
Johnson High School,[District],4761.0,3094650.0,650.0,77.072,80.966,66.058,81.222,53.539,Large (2000-5000)
Pena High School,[Charter],962.0,585858.0,609.0,83.840,84.045,94.595,95.946,90.541,Small (<1000)


In [18]:
# worksheet calcs to find averages in school size groups
size_math_scores = per_school_summary.groupby(["School Size"])["Avg Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Avg Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Passing Overall"].mean()

# concat into summary
size_summary = pd.concat([size_math_scores,size_reading_scores,size_passing_math,size_passing_reading, size_overall_passing], axis=1)
size_summary

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small (<1000),83.821500,83.930000,93.5505,96.0995,89.8840
Medium (1000-2000),83.374400,83.864600,93.5994,96.7908,90.6214
Large (2000-5000),77.746375,81.344375,69.9635,82.7665,58.2860


## Scores by School Type

In [19]:
# not totally sure why it needs to be a str, error otherwise was TypeError: unhashable type: 'numpy.ndarray'
per_school_summary['School Type'] = per_school_summary['School Type'].astype(str)

In [20]:
# same stuff as before, but now we work on school type averages
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Avg Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Avg Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Passing Overall"].mean()

# concat into summary
type_summary = pd.concat([average_math_score_by_type,average_reading_score_by_type,average_percent_passing_math_by_type,average_percent_passing_reading_by_type,average_percent_overall_passing_by_type],axis=1)
type_summary

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Type,,,,,
['Charter'],83.473625,83.896500,93.620750,96.586625,90.432250
['District'],76.956714,80.966571,66.548571,80.798857,53.672143
